In [8]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using BenchmarkTools
using OrderedCollections

## Funzione da ottimizzare

In [9]:
function boxcovering(bboxes, index, tree)
    covers = [[] for k=1:length(bboxes)]
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        for x in iterator
            append!(covers[i],x.value)
        end
    end
    return covers
end

boxcovering (generic function with 1 method)

## 0) Benchmark vecchia funzione 

In [11]:
bboxes = [[ 0.0 1.0; 0.0 0.0],  [ 0.0 0.0 ; 0.0 1.0], 
          [ 0.0 1.0; 0.0 1.0],  [-1.0 0.0 ; 0.0 0.5],
          [-1.0 0.0; 0.5 0.5],  [-1.0 -1.0; 0.0 0.5]]
xdict = OrderedDict([0.0, 1.0] => [1, 3],
                    [0.0, 0.0] => [2],
                    [-1.0, 0.0] => [4, 5],
                    [-1.0, -1.0] => [6])
xs = IntervalTrees.IntervalMap{Float64, Array}()
for (key,boxset) in xdict
    xs[tuple(key...)] = boxset
end
@btime boxcovering(bboxes,1,xs)

  5.085 μs (98 allocations: 3.95 KiB)


6-element Array{Array{Any,1},1}:
 [4, 5, 2, 1, 3]
 [4, 5, 2, 1, 3]
 [4, 5, 2, 1, 3]
 [6, 4, 5, 2, 1, 3]
 [6, 4, 5, 2, 1, 3]
 [6, 4, 5]

## Controllo se la funzione è type unstable

In [12]:
@code_warntype boxcovering(bboxes,1,xs)

Variables
  #self#::Core.Compiler.Const(boxcovering, false)
  bboxes::Array{Array{Float64,2},1}
  index::Int64
  tree::IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}
  #4::var"#4#5"
  covers::Array{Array{Any,1},1}
  @_7::Union{Nothing, Tuple{Tuple{Int64,Array{Float64,2}},Tuple{Int64,Int64}}}
  i::Int64
  boundingbox::Array{Float64,2}
  @_10::Int64
  extent::Array{Float64,1}
  iterator::Union{Array{IntervalValue{Float64,Array},1}, IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp),Float64,IntervalValue{Float64,Array},64}}
  @_13::Union{Nothing, Tuple{IntervalValue{Float64,Array},Union{Nothing, Int64}}}
  x::IntervalValue{Float64,Array}

Body::Array{Array{Any,1},1}
1 ─       (#4 = %new(Main.:(var"#4#5")))
│   %2  = #4::Core.Compiler.Const(var"#4#5"(), false)
│   %3  = Main.length(bboxes)::Int64
│   %4  = (1:%3)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %5  = Base.Generator(%2, %4)::Core.Compi

la funzione NON è type unstable in quanto ho nell'output la stringa:

    Body::Array{Array{Any,1},1}